# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 17 2023 8:30AM,255159,10,0086302951,ISDIN Corporation,Released
1,Jan 17 2023 8:30AM,255159,10,0086302952,ISDIN Corporation,Released
2,Jan 17 2023 8:30AM,255159,10,0086302954,ISDIN Corporation,Released
3,Jan 17 2023 8:30AM,255159,10,0086302953,ISDIN Corporation,Released
4,Jan 17 2023 8:30AM,255159,10,0086302961,ISDIN Corporation,Released
5,Jan 17 2023 8:30AM,255159,10,0086302962,ISDIN Corporation,Released
6,Jan 17 2023 8:30AM,255159,10,0086302965,ISDIN Corporation,Released
7,Jan 17 2023 8:30AM,255159,10,0086302966,ISDIN Corporation,Released
8,Jan 17 2023 8:30AM,255165,10,0086302964,ISDIN Corporation,Released
9,Jan 17 2023 8:30AM,255159,10,0086302963,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
24,255159,Released,9
25,255164,Released,3
26,255165,Released,1
27,255166,Released,1
28,255168,Released,6


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
255159,NaN,NaN,9.0
255164,NaN,NaN,3.0
255165,NaN,NaN,1.0
255166,NaN,NaN,1.0
255168,NaN,NaN,6.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
255049,1.0,36.0,5.0
255053,0.0,0.0,1.0
255087,0.0,1.0,0.0
255090,12.0,21.0,15.0
255096,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
255049,1,36,5
255053,0,0,1
255087,0,1,0
255090,12,21,15
255096,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,255049,1,36,5
1,255053,0,0,1
2,255087,0,1,0
3,255090,12,21,15
4,255096,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,255049,1,36,5
1,255053,,,1
2,255087,,1,
3,255090,12,21,15
4,255096,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 17 2023 8:30AM,255159,10,ISDIN Corporation
8,Jan 17 2023 8:30AM,255165,10,ISDIN Corporation
10,Jan 17 2023 8:30AM,255164,10,ISDIN Corporation
13,Jan 17 2023 8:30AM,255168,10,ISDIN Corporation
19,Jan 17 2023 8:30AM,255155,10,Ivaoes Animal Health
21,Jan 17 2023 8:30AM,255166,10,Emerginnova
22,Jan 16 2023 2:45PM,255142,10,"Methapharm, Inc."
40,Jan 16 2023 2:44PM,255145,19,VITABIOTICS USA INC.
46,Jan 16 2023 2:44PM,255143,10,ISDIN Corporation
60,Jan 16 2023 2:44PM,255146,10,Methapharm-G


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 17 2023 8:30AM,255159,10,ISDIN Corporation,,,9
1,Jan 17 2023 8:30AM,255165,10,ISDIN Corporation,,,1
2,Jan 17 2023 8:30AM,255164,10,ISDIN Corporation,,,3
3,Jan 17 2023 8:30AM,255168,10,ISDIN Corporation,,,6
4,Jan 17 2023 8:30AM,255155,10,Ivaoes Animal Health,,,2
5,Jan 17 2023 8:30AM,255166,10,Emerginnova,,,1
6,Jan 16 2023 2:45PM,255142,10,"Methapharm, Inc.",,,18
7,Jan 16 2023 2:44PM,255145,19,VITABIOTICS USA INC.,,5,1
8,Jan 16 2023 2:44PM,255143,10,ISDIN Corporation,,,14
9,Jan 16 2023 2:44PM,255146,10,Methapharm-G,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 17 2023 8:30AM,255159,10,ISDIN Corporation,9,,
1,Jan 17 2023 8:30AM,255165,10,ISDIN Corporation,1,,
2,Jan 17 2023 8:30AM,255164,10,ISDIN Corporation,3,,
3,Jan 17 2023 8:30AM,255168,10,ISDIN Corporation,6,,
4,Jan 17 2023 8:30AM,255155,10,Ivaoes Animal Health,2,,
5,Jan 17 2023 8:30AM,255166,10,Emerginnova,1,,
6,Jan 16 2023 2:45PM,255142,10,"Methapharm, Inc.",18,,
7,Jan 16 2023 2:44PM,255145,19,VITABIOTICS USA INC.,1,5,
8,Jan 16 2023 2:44PM,255143,10,ISDIN Corporation,14,,
9,Jan 16 2023 2:44PM,255146,10,Methapharm-G,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 17 2023 8:30AM,255159,10,ISDIN Corporation,9,,
1,Jan 17 2023 8:30AM,255165,10,ISDIN Corporation,1,,
2,Jan 17 2023 8:30AM,255164,10,ISDIN Corporation,3,,
3,Jan 17 2023 8:30AM,255168,10,ISDIN Corporation,6,,
4,Jan 17 2023 8:30AM,255155,10,Ivaoes Animal Health,2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 17 2023 8:30AM,255159,10,ISDIN Corporation,9.0,NaN,NaN
1,Jan 17 2023 8:30AM,255165,10,ISDIN Corporation,1.0,NaN,NaN
2,Jan 17 2023 8:30AM,255164,10,ISDIN Corporation,3.0,NaN,NaN
3,Jan 17 2023 8:30AM,255168,10,ISDIN Corporation,6.0,NaN,NaN
4,Jan 17 2023 8:30AM,255155,10,Ivaoes Animal Health,2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 17 2023 8:30AM,255159,10,ISDIN Corporation,9.0,0.0,0.0
1,Jan 17 2023 8:30AM,255165,10,ISDIN Corporation,1.0,0.0,0.0
2,Jan 17 2023 8:30AM,255164,10,ISDIN Corporation,3.0,0.0,0.0
3,Jan 17 2023 8:30AM,255168,10,ISDIN Corporation,6.0,0.0,0.0
4,Jan 17 2023 8:30AM,255155,10,Ivaoes Animal Health,2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3827132,109.0,1.0,0.0
12,255115,1.0,0.0,0.0
15,255049,5.0,36.0,1.0
19,1275597,17.0,28.0,12.0
21,255087,0.0,1.0,0.0
22,510149,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3827132,109.0,1.0,0.0
1,12,255115,1.0,0.0,0.0
2,15,255049,5.0,36.0,1.0
3,19,1275597,17.0,28.0,12.0
4,21,255087,0.0,1.0,0.0
5,22,510149,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,109.0,1.0,0.0
1,12,1.0,0.0,0.0
2,15,5.0,36.0,1.0
3,19,17.0,28.0,12.0
4,21,0.0,1.0,0.0
5,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,109.0
1,12,Released,1.0
2,15,Released,5.0
3,19,Released,17.0
4,21,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,21,22
Status,,,,,,
Completed,0.0,0.0,1.0,12.0,0.0,0.0
Executing,1.0,0.0,36.0,28.0,1.0,0.0
Released,109.0,1.0,5.0,17.0,0.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,21,22
0,Completed,0.0,0.0,1.0,12.0,0.0,0.0
1,Executing,1.0,0.0,36.0,28.0,1.0,0.0
2,Released,109.0,1.0,5.0,17.0,0.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,21,22
0,Completed,0.0,0.0,1.0,12.0,0.0,0.0
1,Executing,1.0,0.0,36.0,28.0,1.0,0.0
2,Released,109.0,1.0,5.0,17.0,0.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()